# LAB 1 Collecting the Data

In [3]:
# Requests allows us to make HTTP requests which we will use to get data from an API
import requests
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Datetime is a library that allows us to represent dates
import datetime

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

In [63]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])

In [62]:
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])

In [6]:
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])

In [7]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

In [8]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

In [9]:
response = requests.get(spacex_url)

In [ ]:
print(response.content)

# Task 1: Request and parse the SpaceX launch data using the GET request


In [11]:
static_json_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'

In [12]:
response=requests.get(static_json_url)

In [13]:
response.status_code

200

In [14]:
data = response.json()

In [15]:
df = pd.json_normalize(data)
print(df.head(5))

       static_fire_date_utc  static_fire_date_unix    tbd    net  window  \
0  2006-03-17T00:00:00.000Z           1.142554e+09  False  False     0.0   
1                      None                    NaN  False  False     0.0   
2                      None                    NaN  False  False     0.0   
3  2008-09-20T00:00:00.000Z           1.221869e+09  False  False     0.0   
4                      None                    NaN  False  False     0.0   

                     rocket  success  \
0  5e9d0d95eda69955f709d1eb    False   
1  5e9d0d95eda69955f709d1eb    False   
2  5e9d0d95eda69955f709d1eb    False   
3  5e9d0d95eda69955f709d1eb     True   
4  5e9d0d95eda69955f709d1eb     True   

                                                                                                                                                                                details  \
0                                                                                                                  

In [57]:
# First, create the DataFrame from the JSON data
df = pd.json_normalize(data)

# Select the subset of columns we want
df_subset = df[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# Remove rows with multiple cores and multiple payloads
df_subset = df_subset[df_subset['cores'].map(len)==1]
df_subset = df_subset[df_subset['payloads'].map(len)==1]

# Extract single values from lists
df_subset['cores'] = df_subset['cores'].map(lambda x : x[0])
df_subset['payloads'] = df_subset['payloads'].map(lambda x : x[0])

# Convert date_utc to datetime and extract date
df_subset['date'] = pd.to_datetime(df_subset['date_utc']).dt.date

# Restrict dates of launches
df_subset = df_subset[df_subset['date'] <= datetime.date(2020, 11, 13)]

In [56]:
# Load JSON
static_json_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'
response = requests.get(static_json_url)
data = response.json()

# Normalize into DataFrame
df = pd.json_normalize(data)

# Lets take a subset of our dataframe keeping only the features we want and the flight number, and date_utc.
df = df[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# We will remove rows with multiple cores because those are falcon rockets with 2 extra rocket boosters and rows that have multiple payloads in a single rocket.
df = df[df['cores'].map(len) == 1]
df = df[df['payloads'].map(len) == 1]

# Since payloads and cores are lists of size 1 we will also extract the single value in the list and replace the feature
df['cores'] = df['cores'].map(lambda x: x[0])
df['payloads'] = df['payloads'].map(lambda x: x[0])

# We also want to convert the date_utc to a datetime datatype and then extracting the date leaving the time
df['date'] = pd.to_datetime(df['date_utc']).dt.date

# Using the date we will restrict the dates of the launches
df = df[df['date'] <= datetime.date(2020, 11, 13)]

print(df.head())

                     rocket                  payloads  \
0  5e9d0d95eda69955f709d1eb  5eb0e4b5b6c3bb0006eeb1e1   
1  5e9d0d95eda69955f709d1eb  5eb0e4b6b6c3bb0006eeb1e2   
3  5e9d0d95eda69955f709d1eb  5eb0e4b7b6c3bb0006eeb1e5   
4  5e9d0d95eda69955f709d1eb  5eb0e4b7b6c3bb0006eeb1e6   
5  5e9d0d95eda69973a809d1ec  5eb0e4b7b6c3bb0006eeb1e7   

                  launchpad  \
0  5e9e4502f5090995de566f86   
1  5e9e4502f5090995de566f86   
3  5e9e4502f5090995de566f86   
4  5e9e4502f5090995de566f86   
5  5e9e4501f509094ba4566f84   

                                                                                                                                                                                            cores  \
0  {'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}   
1  {'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'leg

In [58]:
#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

In [59]:
BoosterVersion

[]

In [31]:
# Call getBoosterVersion
# def getBoosterVersion(data):
    #for x in data['rocket']:
       # response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        #BoosterVersion.append(response['name'])
#getBoosterVersion(data)
#This code doesn't seem to work due to data being a df and data'rocket' is panda series not a list of IDs, or data'rocket' doesn'ts have string keys becainse its a list
# I don't know it yet, but I will get it.

In [55]:
# Call getBoosterVersion
def getBoosterVersion(df):
    booster_versions = []
    for rocket_id in df['rocket']:
        if rocket_id:  # make sure it's not None
            response = requests.get("https://api.spacexdata.com/v4/rockets/" + str(rocket_id)).json()
            booster_versions.append(response['name'])
    return booster_versions

# Example usage:
df['BoosterVersion'] = getBoosterVersion(df)
print(df[['rocket', 'BoosterVersion']].head())

# thise code loops though each rocket ID in the dataframe
# calls the spaceX API endpoint
# extracts the rocket's name
#  Always work with df (your DataFrame) after normalization, not the raw data list. That way, column access like df['rocket'] works correctly.

                     rocket BoosterVersion
0  5e9d0d95eda69955f709d1eb       Falcon 1
1  5e9d0d95eda69955f709d1eb       Falcon 1
2  5e9d0d95eda69955f709d1eb       Falcon 1
3  5e9d0d95eda69955f709d1eb       Falcon 1
4  5e9d0d95eda69973a809d1ec       Falcon 9


In [67]:
BoosterVersion[0:5]

[]

In [78]:
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])

In [42]:
def getPayloadData(df):
    # Changed: define lists inside the function
    # Reason: avoids reliance on undefined global variables and makes the function self-contained
    payload_mass = []
    orbit = []
    
    # Changed: iterate over df['payloads'] which contains payload IDs
    # Reason: ensures we are looping through the correct column in the DataFrame
    for payload_id in df['payloads']:
        if pd.notna(payload_id):  # Added: skip missing values safely
            # Changed: use f-string for URL construction
            # Reason: f-strings are cleaner and faster than string concatenation
            response = requests.get(f"https://api.spacexdata.com/v4/payloads/{payload_id}").json()
            
            # Changed: use .get() for dictionary access
            # Reason: prevents errors if a key is missing in the API response
            payload_mass.append(response.get('mass_kg'))
            orbit.append(response.get('orbit'))
        else:
            # Added: append None when payload_id is missing
            # Reason: ensures list lengths remain consistent with DataFrame rows
            payload_mass.append(None)
            orbit.append(None)
    
    # Changed: return a DataFrame instead of modifying global lists
    # Reason: improves modularity and makes merging with the main DataFrame straightforward
    return pd.DataFrame({
        'PayloadMass': payload_mass,
        'Orbit': orbit
    })

# Example usage
payload_info = getPayloadData(df)

# Changed: use pd.concat to merge results back into the DataFrame
# Reason: keeps rows aligned correctly without manual indexing
df = pd.concat([df.reset_index(drop=True), payload_info], axis=1)

print(df[['payloads', 'PayloadMass', 'Orbit']].head())

                   payloads  PayloadMass Orbit
0  5eb0e4b5b6c3bb0006eeb1e1         20.0   LEO
1  5eb0e4b6b6c3bb0006eeb1e2          NaN   LEO
2  5eb0e4b7b6c3bb0006eeb1e5        165.0   LEO
3  5eb0e4b7b6c3bb0006eeb1e6        200.0   LEO
4  5eb0e4b7b6c3bb0006eeb1e7          NaN   LEO


In [43]:
def getCoreData(df):
    # Changed: define lists inside the function
    # Reason: avoids reliance on undefined global variables and makes the function self-contained
    block, reused_count, serial = [], [], []
    outcome, flights, gridfins = [], [], []
    reused, legs, landing_pad = [], [], []
    
    # Changed: iterate over df['cores'] which contains dictionaries
    # Reason: ensures we are looping through the correct column in the DataFrame
    for core in df['cores']:
        if core and core.get('core') is not None:  # Added: safe check for None values
            # Changed: use f-string for URL construction
            # Reason: f-strings are cleaner and faster than string concatenation
            response = requests.get(f"https://api.spacexdata.com/v4/cores/{core['core']}").json()
            
            # Changed: use .get() for dictionary access
            # Reason: prevents errors if a key is missing in the API response
            block.append(response.get('block'))
            reused_count.append(response.get('reuse_count'))
            serial.append(response.get('serial'))
        else:
            # Added: append None when core ID is missing
            # Reason: ensures list lengths remain consistent with DataFrame rows
            block.append(None)
            reused_count.append(None)
            serial.append(None)
        
        # Changed: use .get() for dictionary access on core fields
        # Reason: prevents KeyError if a field is missing in the core dictionary
        outcome.append(str(core.get('landing_success')) + ' ' + str(core.get('landing_type')))
        flights.append(core.get('flight'))
        gridfins.append(core.get('gridfins'))
        reused.append(core.get('reused'))
        legs.append(core.get('legs'))
        landing_pad.append(core.get('landpad'))
    
    # Changed: return a DataFrame instead of modifying global lists
    # Reason: improves modularity and makes merging with the main DataFrame straightforward
    return pd.DataFrame({
        'Block': block,
        'ReusedCount': reused_count,
        'Serial': serial,
        'Outcome': outcome,
        'Flights': flights,
        'GridFins': gridfins,
        'Reused': reused,
        'Legs': legs,
        'LandingPad': landing_pad
    })

# Example usage
core_info = getCoreData(df)

# Changed: use pd.concat to merge results back into the DataFrame
# Reason: keeps rows aligned correctly without manual indexing
df = pd.concat([df.reset_index(drop=True), core_info], axis=1)

print(df[['cores', 'Block', 'ReusedCount', 'Serial', 'Outcome']].head())

                                                                                                                                                                                            cores  \
0  {'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}   
1  {'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}   
2  {'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}   
3  {'core': '5e9e289ef359184f103b2627', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}   
4  {'core': '5e

In [5]:
# First, define the df variable by loading your data
import pandas as pd
import requests

# Load your SpaceX data - replace this URL with your actual data source if different
df = pd.read_json('https://api.spacexdata.com/v4/launches')

# Define the getCoreData function
def getCoreData(data):
    # Initialize empty lists to store core data
    block = []
    reused_count = []
    serial = []
    outcome = []
    flights = []
    gridfins = []
    reused = []
    legs = []
    landing_pad = []
    
    # Extract core information from each launch in the DataFrame
    for core in data['cores']:
        if core and 'core' in core.keys():  # Added: safe check for None values
            # Changed: use f-string for URL construction
            # Reason: f-strings are cleaner and faster than string concatenation
            response = requests.get(f"https://api.spacexdata.com/v4/cores/{core['core']}").json()
            
            # Changed: use .get() for dictionary access
            block.append(response.get('block'))
            reused_count.append(response.get('reuse_count'))
            serial.append(response.get('serial'))
        else:
            # Added: append None when core ID is missing
            # Reason: ensures list lengths remain consistent with DataFrame rows
            block.append(None)
            reused_count.append(None)
            serial.append(None)
        
        # Changed: use .get() for dictionary access on core fields
        outcome.append(str(core.get('landing_success')) + ' ' + str(core.get('landing_type')))
        flights.append(core.get('flight'))
        gridfins.append(core.get('gridfins'))
        reused.append(core.get('reused'))
        legs.append(core.get('legs'))
        landing_pad.append(core.get('landpad'))
    
    # Changed: return a DataFrame instead of modifying global lists
    # Reason: improves modularity and makes merging with the main DataFrame straightforward
    return pd.DataFrame({
        'Block': block,
        'ReusedCount': reused_count,
        'Serial': serial,
        'Outcome': outcome,
        'Flights': flights,
        'GridFins': gridfins,
        'Reused': reused,
        'Legs': legs,
        'LandingPad': landing_pad
    })

# Example usage
core_info = getCoreData(df)

# Changed: use pd.concat to merge results back into the DataFrame
# Reason: keeps rows aligned correctly without manual indexing
df = pd.concat([df.reset_index(drop=True), core_info], axis=1)

print(df[['cores', 'Block', 'ReusedCount', 'Serial', 'Outcome']].head())

AttributeError: 'list' object has no attribute 'keys'

In [47]:
#anaconda AI correction version:

def getBoosterVersion(data):
    """
    Extract booster version information from launch data.
    
    Parameters:
    data (list): List of launch dictionaries
    
    Returns:
    list: List of booster versions corresponding to each launch
    """
    # Initialize an empty list to store booster versions
    BoosterVersion = []
    
    # Iterate through each launch in the data
    for launch in data:
        try:
            # Check if rocket ID exists and is not None
            if 'rocket' in launch and launch['rocket']:
                # Make API request to get rocket details
                response = requests.get("https://api.spacexdata.com/v4/rockets/" + str(launch['rocket']))
                
                # Check if request was successful
                if response.status_code == 200:
                    rocket_data = response.json()
                    BoosterVersion.append(rocket_data['name'])
                else:
                    # Handle API error by adding a placeholder
                    BoosterVersion.append("Unknown")
            else:
                # Handle missing rocket data
                BoosterVersion.append("Unknown")
        except Exception as e:
            # Handle any exceptions and continue processing
            print(f"Error processing rocket data: {e}")
            BoosterVersion.append("Error")
    
    # Return the list of booster versions
    return BoosterVersion

In [75]:
# First, let's check what columns are available in the DataFrame
print("Available columns in df:", df.columns.tolist())

# Based on the available columns, construct the dictionary
# For now, let's comment out the problematic line
launch_dict = {
    # 'FlightNumber': list(df['flight_number']),  # Commented out until we know the correct column name
    'Date': list(df['date']),                    # launch dates
    'BoosterVersion': BoosterVersion,            # extracted booster versions
    'PayloadMass': PayloadMass,                  # payload mass values
    'Orbit': Orbit,                              # orbit types
    'LaunchSite': LaunchSite,                    # launch site names
    'Outcome': Outcome,                          # landing outcomes
    'Flights': Flights,                          # number of flights
    'GridFins': GridFins,                        # grid fins status
    'Reused': Reused,                            # reused status
    'Legs': Legs,                                # landing legs status
    'LandingPad': LandingPad,                    # landing pad IDs
    'Block': Block,                              # block numbers
    'ReusedCount': ReusedCount,                  # reuse counts
    'Serial': Serial,                            # core serials
    'Longitude': Longitude,                      # launch site longitude
    'Latitude': Latitude                         # launch site latitude
}

# After seeing the available columns, uncomment and update the FlightNumber line with the correct column name
# launch_dict['FlightNumber'] = list(df['correct_column_name'])

# Convert dictionary into a DataFrame
launch_df = pd.DataFrame(launch_dict)

# Preview the dataset
print(launch_df.head())

Available columns in df: ['payload_weights', 'flickr_images', 'name', 'type', 'active', 'stages', 'boosters', 'cost_per_launch', 'success_rate_pct', 'first_flight', 'country', 'company', 'wikipedia', 'description', 'id', 'height.meters', 'height.feet', 'diameter.meters', 'diameter.feet', 'mass.kg', 'mass.lb', 'first_stage.thrust_sea_level.kN', 'first_stage.thrust_sea_level.lbf', 'first_stage.thrust_vacuum.kN', 'first_stage.thrust_vacuum.lbf', 'first_stage.reusable', 'first_stage.engines', 'first_stage.fuel_amount_tons', 'first_stage.burn_time_sec', 'second_stage.thrust.kN', 'second_stage.thrust.lbf', 'second_stage.payloads.composite_fairing.height.meters', 'second_stage.payloads.composite_fairing.height.feet', 'second_stage.payloads.composite_fairing.diameter.meters', 'second_stage.payloads.composite_fairing.diameter.feet', 'second_stage.payloads.option_1', 'second_stage.reusable', 'second_stage.engines', 'second_stage.fuel_amount_tons', 'second_stage.burn_time_sec', 'engines.isp.sea_l

KeyError: 'date'

   FlightNumber        Date BoosterVersion  PayloadMass Orbit LaunchSite  \
0             1  2006-03-24                           0                    
1             2  2007-03-21                           0                    
2             4  2008-09-28                           0                    
3             5  2009-07-13                           0                    
4             6  2010-06-04                           0                    

  Outcome  Flights  GridFins  Reused   Legs LandingPad  Block  ReusedCount  \
0                0     False   False  False                 0            0   
1                0     False   False  False                 0            0   
2                0     False   False  False                 0            0   
3                0     False   False  False                 0            0   
4                0     False   False  False                 0            0   

  Serial  Longitude  Latitude  
0               0.0       0.0  
1         

In [48]:
# You can also check how many rows were filtered out
filtered_df = launch_df[launch_df['BoosterVersion'] != 'Falcon 1']
print(f"Original DataFrame size: {len(launch_df)}")
print(f"Filtered DataFrame size: {len(filtered_df)}")

Original DataFrame size: 94
Filtered DataFrame size: 94


In [50]:
# First, filter out Falcon 1 rockets
data_falcon9 = launch_df[launch_df['BoosterVersion'] != 'Falcon 1']

# Then reset the FlightNumber column to sequential numbers
data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))

# Display the result
data_falcon9

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2006-03-24,,0,,,,0,False,False,False,,0,0,,0.0,0.0
1,2,2007-03-21,,0,,,,0,False,False,False,,0,0,,0.0,0.0
2,3,2008-09-28,,0,,,,0,False,False,False,,0,0,,0.0,0.0
3,4,2009-07-13,,0,,,,0,False,False,False,,0,0,,0.0,0.0
4,5,2010-06-04,,0,,,,0,False,False,False,,0,0,,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,90,2020-09-03,,0,,,,0,False,False,False,,0,0,,0.0,0.0
90,91,2020-10-06,,0,,,,0,False,False,False,,0,0,,0.0,0.0
91,92,2020-10-18,,0,,,,0,False,False,False,,0,0,,0.0,0.0
92,93,2020-10-24,,0,,,,0,False,False,False,,0,0,,0.0,0.0


In [51]:
data_falcon9.isnull().sum()

FlightNumber      0
Date              0
BoosterVersion    0
PayloadMass       0
Orbit             0
LaunchSite        0
Outcome           0
Flights           0
GridFins          0
Reused            0
Legs              0
LandingPad        0
Block             0
ReusedCount       0
Serial            0
Longitude         0
Latitude          0
dtype: int64

In [53]:
# Only calculate the mean of the PayloadMass column directly
payload_mean = data_falcon9['PayloadMass'].mean()

# Print the mean value
print(f"Mean payload mass: {payload_mean}")

# Replace NaN values in PayloadMass column with the calculated mean
data_falcon9['PayloadMass'] = data_falcon9['PayloadMass'].replace(np.nan, payload_mean)

# Verify there are no more NaN values in the PayloadMass column
print(f"Number of NaN values after replacement: {data_falcon9['PayloadMass'].isna().sum()}")

# Display the first few rows to verify the changes
print(data_falcon9[['FlightNumber', 'PayloadMass']].head())

Mean payload mass: 0.0
Number of NaN values after replacement: 0
   FlightNumber  PayloadMass
0             1            0
1             2            0
2             3            0
3             4            0
4             5            0


### After inspecting all the blocks, here is the complete code with the last task to test if I got everything right.

In [10]:
import requests
import pandas as pd
import numpy as np
import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Initialize lists once
BoosterVersion, Longitude, Latitude, LaunchSite = [], [], [], []
PayloadMass, Orbit = [], []
Block, ReusedCount, Serial = [], [], []
Outcome, Flights, GridFins, Reused, Legs, LandingPad = [], [], [], [], [], []

def getBoosterVersion(df):
    for x in df['rocket']:
        if x:
            response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
            BoosterVersion.append(response['name'])

def getLaunchSite(df):
    for x in df['launchpad']:
        if x:
            response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
            Longitude.append(response['longitude'])
            Latitude.append(response['latitude'])
            LaunchSite.append(response['name'])

def getPayloadData(df):
    for load in df['payloads']:
        if load:
            response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
            PayloadMass.append(response['mass_kg'])
            Orbit.append(response['orbit'])

def getCoreData(df):
    for core in df['cores']:
        if core['core'] is not None:
            response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
            Block.append(response['block'])
            ReusedCount.append(response['reuse_count'])
            Serial.append(response['serial'])
        else:
            Block.append(None)
            ReusedCount.append(None)
            Serial.append(None)
        Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
        Flights.append(core['flight'])
        GridFins.append(core['gridfins'])
        Reused.append(core['reused'])
        Legs.append(core['legs'])
        LandingPad.append(core['landpad'])

# Load dataset
static_json_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'
data = requests.get(static_json_url).json()

# Normalize into DataFrame
df = pd.json_normalize(data)
df = df[['rocket','payloads','launchpad','cores','flight_number','date_utc']]

# Clean
df = df[df['cores'].map(len)==1]
df = df[df['payloads'].map(len)==1]
df['cores'] = df['cores'].map(lambda x : x[0])
df['payloads'] = df['payloads'].map(lambda x : x[0])
df['date'] = pd.to_datetime(df['date_utc']).dt.date
df = df[df['date'] <= datetime.date(2020, 11, 13)]

# Enrich
getBoosterVersion(df)
getLaunchSite(df)
getPayloadData(df)
getCoreData(df)

# Build final DataFrame
launch_dict = {
    'FlightNumber': list(df['flight_number']),
    'Date': list(df['date']),
    'BoosterVersion': BoosterVersion,
    'PayloadMass': PayloadMass,
    'Orbit': Orbit,
    'LaunchSite': LaunchSite,
    'Outcome': Outcome,
    'Flights': Flights,
    'GridFins': GridFins,
    'Reused': Reused,
    'Legs': Legs,
    'LandingPad': LandingPad,
    'Block': Block,
    'ReusedCount': ReusedCount,
    'Serial': Serial,
    'Longitude': Longitude,
    'Latitude': Latitude
}

launch_df = pd.DataFrame(launch_dict)
print(launch_df.head()) 

   FlightNumber        Date BoosterVersion  PayloadMass Orbit  \
0             1  2006-03-24       Falcon 1         20.0   LEO   
1             2  2007-03-21       Falcon 1          NaN   LEO   
2             4  2008-09-28       Falcon 1        165.0   LEO   
3             5  2009-07-13       Falcon 1        200.0   LEO   
4             6  2010-06-04       Falcon 9          NaN   LEO   

        LaunchSite    Outcome  Flights  GridFins  Reused   Legs LandingPad  \
0  Kwajalein Atoll  None None        1     False   False  False       None   
1  Kwajalein Atoll  None None        1     False   False  False       None   
2  Kwajalein Atoll  None None        1     False   False  False       None   
3  Kwajalein Atoll  None None        1     False   False  False       None   
4     CCSFS SLC 40  None None        1     False   False  False       None   

   Block  ReusedCount    Serial   Longitude   Latitude  
0    NaN            0  Merlin1A  167.743129   9.047721  
1    NaN            0  Mer

In [11]:
# Filter out Falcon 1 launches, keeping only Falcon 9
data_falcon9 = launch_df[launch_df['BoosterVersion'] == 'Falcon 9']

# Reset FlightNumber to be sequential starting at 1
data_falcon9.loc[:, 'FlightNumber'] = list(range(1, data_falcon9.shape[0] + 1))

# Display the cleaned Falcon 9 dataset
print(data_falcon9.head())

   FlightNumber        Date BoosterVersion  PayloadMass Orbit    LaunchSite  \
4             1  2010-06-04       Falcon 9          NaN   LEO  CCSFS SLC 40   
5             2  2012-05-22       Falcon 9        525.0   LEO  CCSFS SLC 40   
6             3  2013-03-01       Falcon 9        677.0   ISS  CCSFS SLC 40   
7             4  2013-09-29       Falcon 9        500.0    PO   VAFB SLC 4E   
8             5  2013-12-03       Falcon 9       3170.0   GTO  CCSFS SLC 40   

       Outcome  Flights  GridFins  Reused   Legs LandingPad  Block  \
4    None None        1     False   False  False       None    1.0   
5    None None        1     False   False  False       None    1.0   
6    None None        1     False   False  False       None    1.0   
7  False Ocean        1     False   False  False       None    1.0   
8    None None        1     False   False  False       None    1.0   

   ReusedCount Serial   Longitude   Latitude  
4            0  B0003  -80.577366  28.561857  
5         

In [12]:
data_falcon9.isnull().sum()


FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        5
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

In [13]:
payload_mean = launch_df['PayloadMass'].mean()
launch_df['PayloadMass'].fillna(payload_mean, inplace=True)
print(launch_df['PayloadMass'].isnull().sum())

0


C:\Users\heinr\AppData\Local\Temp\ipykernel_16252\553081754.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  launch_df['PayloadMass'].fillna(payload_mean, inplace=True)


In [14]:
data_falcon9.to_csv('dataset_part_1.csv', index=False)
